In [393]:
import numpy as np
import pandas as pd
df=pd.read_csv('df_preg_hormone_.csv')

In [394]:
print(np.__version__)

1.12.1


In [395]:
df.dropna(subset=['TVAL_CHAR', 'NVAL_NUM'], how='all')
print(df.shape)

(87384, 12)


In [396]:
print(df.shape)
# df.describe(include='all')

(87384, 12)


In [397]:
# df.head()

In [398]:
len(df.PATIENT_NUM.unique())

16850

# only keep patients with hormone and delete patients with only preg

In [399]:
keep_patient=[]
for id in df.PATIENT_NUM.unique():#:#[0:1]:
#     patient=df[df.PATIENT_NUM==id]
    if sum(~df[df.PATIENT_NUM==id].CONCEPT_CD.isin( ['HDID:2578','HDID:50588']))>0:
        keep_patient.append(id)
    
print(len(keep_patient))

df=df[df.PATIENT_NUM.isin(keep_patient)]

5933


In [400]:
# df#.head()

In [401]:
len(df.PATIENT_NUM.unique())

5933

In [402]:
# df.CONCEPT_CD.value_counts().to_frame()

# delete irregular preg result in HDID:2578 and 'HDID:50588'

In [403]:
df.loc[df.CONCEPT_CD.isin( ['HDID:2578','HDID:50588']),'TVAL_CHAR']=df[df.CONCEPT_CD.isin( ['HDID:2578','HDID:50588'])].TVAL_CHAR.str.lower()

In [404]:
name2578=df[df.CONCEPT_CD.isin( ['HDID:2578','HDID:50588'])].TVAL_CHAR.unique()#.str.lower().unique()
#name2578

# name2578.dtype
type(name2578)

name2578=name2578.astype(str)

# Ms') for s in name2578] 

pos_id=[i for i,item in enumerate(name2578) if "pos" in item or " po" in item]
name2578pos= name2578[pos_id]

is_pos=np.zeros(name2578.shape)#,dtype=bool
is_pos[pos_id] = True

neg_id=[i for i,item in enumerate(name2578) if "neg" in item or " ne" in item or 'neative' in item]
name2578neg=name2578[neg_id]

is_neg=np.zeros(name2578.shape)#,dtype=bool
is_neg[neg_id] = True

delete_name2578=name2578[is_pos+is_neg==0]

delete_name2578

array(['n/a', 'done', 'xxxxx', '9000 on', 'ucg: 5/19/', 'urine preg', 'e',
       '+ ucg: 3/1', 'ucg:', 'hcg done n', 'sent 1/30/', '6/21/02 ur',
       'ucg today', 'nan', 'urine hcg', '2017-08-01 00:00:00',
       '2017-07-01 00:00:00', '2017-07-02 00:00:00', '2005-10-01 00:00:00',
       '2017-06-02 00:00:00', 'home ucg +', '2017-05-01 00:00:00', 'f/u',
       'less than 2', 'hyst', '+', '+ ucg: 12/', 'ng ucg tod',
       '2000-08-01 00:00:00', 'ucg +, ucg', 'pt decline', 'not perfor',
       '2001-12-01 00:00:00', '166 on 3/2', '2017-11-20 00:00:00',
       '110/01', '10/2/03 do', 'bhcg: less', '2017-10-01 00:00:00', 'ucg/',
       '(-)', 'ucg: 5/13/', 'ucg/ quest', '3 wks ago', 'ucg done n', '0.5',
       '2017-02-02 00:00:00', '29386', '2017-04-01 00:00:00', 'menopausal',
       'ucg: msb r', '2017-11-02 00:00:00', '2017-11-01 00:00:00',
       'bhcg 1310', 's/p nsvd o', '+ ucg: 8/2', '10003', '24000',
       '10/21/02--', 'urine', 'reported a', 'not sexual', 'not done',
       '

In [405]:
df=df[~df.TVAL_CHAR.isin(delete_name2578)]#.CONCEPT_CD.unique()

# # temp=[i for i,item in enumerate(delete_name2578) if " ne" in item or 'neative' in item]
# # delete_name2578[temp]
# temp=[i for i,item in enumerate(delete_name2578) if " po" in item or 'neative' in item]
# delete_name2578[temp]
print(len(df.PATIENT_NUM.unique()))

# df.loc[np.logical_and (df.CONCEPT_CD=='HDID:2578',df['TVAL_CHAR'].isin(name2578pos)),'TVAL_CHAR']="pos" 

# df.loc[np.logical_and (df.CONCEPT_CD=='HDID:2578',df['TVAL_CHAR'].isin(name2578neg)),'TVAL_CHAR']="neg"
df.loc[df['TVAL_CHAR'].isin(name2578pos),'TVAL_CHAR']="pos" 

df.loc[df['TVAL_CHAR'].isin(name2578neg),'TVAL_CHAR']="neg"

df_preg=df.CONCEPT_CD.isin( ['HDID:2578','HDID:50588'])
df[df_preg].TVAL_CHAR.value_counts()

5933


neg    17669
pos     2865
Name: TVAL_CHAR, dtype: int64

In [406]:
# df[df_preg].TVAL_CHAR.isnull().sum()
# df[df_preg].TVAL_CHAR.isnull().index
drop_row=df[df_preg].loc[df[df_preg].TVAL_CHAR.isnull(),:].index

# drop_row
df=df.drop(drop_row)

# remove records until the last record is HDID:2578 or HDID:50588

In [408]:
drop_row=[]
for id in df.PATIENT_NUM.unique():#[0:1]:
    patient=df[df.PATIENT_NUM==id]
#     print(patient)
    k=len(patient)-1#RECORD_ID
    while k!=-1: 
#         print(k)
        if (patient.iloc[k,:].CONCEPT_CD != 'HDID:2578' and patient.iloc[k,:].CONCEPT_CD != 'HDID:50588') or k == 0:
            drop_row.append(patient.index[k])#patient.iloc[k,0]
            k=k-1
        else:
            break
    
print(len(drop_row))

446


In [409]:
df=df.drop(drop_row)
len(df.PATIENT_NUM.unique())

5717

In [410]:
# df

In [411]:
# np.logical_and (df.CONCEPT_CD=='HDID:2578',df['TVAL_CHAR'].isin(name2578pos))

# only keep patients with hormone and delete patients with only preg (2)

In [413]:
keep_patient=[]
for id in df.PATIENT_NUM.unique():#:#[0:1]:
    patient=df[df.PATIENT_NUM==id]
    if sum(~patient.CONCEPT_CD.isin( ['HDID:2578','HDID:50588']))>0 or sum(patient.CONCEPT_CD.isin( ['HDID:2578','HDID:50588']))==0:
        keep_patient.append(id)
    
print(len(keep_patient))

df=df[df.PATIENT_NUM.isin(keep_patient)]

5712


# only keep last record besides the last preg to be predicted

In [414]:
keep_row=[]
for id in df.PATIENT_NUM.unique():#[0:1]:#[0:1]:
    patient=df[df.PATIENT_NUM==id].iloc[:-1,:]
# #     print(patient)
#     k=len(patient)#RECORD_ID
    keep_row.append(df[df.PATIENT_NUM==id].index[-1])
#     for CD in patient.CONCEPT_CD.unique():
#         #prin​keep_row=[]
# for id in df.PATIENT_NUM.unique():#[0:1]:#[0:1]:
#     patient=df[df.PATIENT_NUM==id].iloc[:-1,:]
# # #     print(patient)
# #     k=len(patient)#RECORD_ID
#     keep_row.append(df[df.PATIENT_NUM==id].index[-1])
    for CD in patient.CONCEPT_CD.unique():
        #print(CD)
        keep_row.append(patient[patient.CONCEPT_CD==CD].index[-1])
    
# print(len(keep_row))t(CD)
#         keep_row.append(patient[patient.CONCEPT_CD==CD].index[-1])
    
# print(len(keep_row))

20683


In [415]:
# keep_row
df2=df.loc[keep_row,:]
df2=df2.sort_values(['PATIENT_NUM','START_DATE'])
# df2#.head()

In [416]:
len(df2.PATIENT_NUM.unique())

5712

# drop last record (preg) if the last 2nd  (preg)  has the same START_DATE

In [417]:
drop_row=[]
for id in df2.PATIENT_NUM.unique():#:#[0:1]:  [0:10]
    patient=df2[df2.PATIENT_NUM==id].iloc[-2:,:]
    if (len(patient.START_DATE.unique()) == 1) and sum(patient.CONCEPT_CD.isin( ['HDID:2578','HDID:50588']))==2:
#         print(patient.START_DATE.unique())
        drop_row.append(patient.tail(1).index[0])
    
print(len(drop_row))

90


In [418]:
df=df2.drop(drop_row)
len(df.PATIENT_NUM.unique())

5712

In [419]:
df[df.CONCEPT_CD.isin( ['HDID:2578','HDID:50588'])].TVAL_CHAR.value_counts().to_frame()

,TVAL_CHAR
neg,8044
pos,1502


In [420]:
#df.NAME_CHAR.value_counts().to_frame()
df.CONCEPT_PATH.value_counts().to_frame()

,CONCEPT_PATH
"\HDID2\Basic\Dx\Lab\Chemistry\human chorionic gonadotropin, urine, qualitative (urine pregnancy test) (HDID:2578)\",9057
"\HDID2\Basic\Dx\Lab\Chemistry\follicle stimulating hormone, serum (HDID:149)\",3571
"\HDID2\Basic\Dx\Lab\Chemistry\luteinizing hormone, serum (HDID:301)\",2677
"\HDID2\Basic\Hx\PMH\infertility, hx of (HDID:7177)\",1637
"\i2b2\Diagnoses\Genitourinary system (580-629)\Other disorders of female genital tract (617-629)\(628) Infertility, female\(628-9) Infertility, female, of u~\",1026
"\HDID2\Basic\Dx\Lab\Chemistry\progesterone, serum (HDID:292)\",728
\SVC2\BMC\OP\OB/GYN\OB/GYN-FAMILY PLANNING\,507
\HDID2\Basic\Dx\Lab\Genetics/fertility\pregnancy test results (HDID:50588)\,489
"\HDID2\Basic\Dx\Lab\Chemistry\17-hydroxyprogesterone, serum or plasma (HDID:3232)\",346
"\i2b2\Diagnoses\Genitourinary system (580-629)\Other disorders of female genital tract (617-629)\(628) Infertility, female\(628-8) Infertility, female, of o~\",328


In [421]:
df.CONCEPT_CD.value_counts().to_frame()

,CONCEPT_CD
HDID:2578,9057
HDID:149,3571
HDID:301,2677
HDID:7177,1637
ICD9:628.9,1026
HDID:292,728
SVC:BMC:OB/GYN-FAMILY PLANNING,507
HDID:50588,489
HDID:3232,346
ICD9:628.8,328


In [422]:
print(df.shape)
df.describe(include='all')

(20593, 12)


,PATIENT_NUM,CONCEPT_CD,START_DATE,VALTYPE_CD,TVAL_CHAR,NVAL_NUM,CONCEPT_PATH,NAME_CHAR,SEX_CD,LANGUAGE_CD,RACE_CD,ZIP_CD
count,2.059300e+04,20593,20593,20593,20593,7056.000000,20593,20593,20593,20593,20593,20590
unique,NaN,11,17750,2,107,NaN,11,11,1,25,12,322
top,NaN,HDID:2578,2010-08-23 15:26:00,T,neg,NaN,\HDID2\Basic\Dx\Lab\Chemistry\human chorionic ...,"human chorionic gonadotropin, urine, qualitati...",F,DEM|LANGUAGE:engl,DEM|RACE:black,2124
freq,NaN,9057,4,13537,9600,NaN,9057,9057,20593,15470,12364,1864
mean,2.915714e+06,NaN,NaN,NaN,NaN,26.887826,NaN,NaN,NaN,NaN,NaN,NaN
std,2.376369e+06,NaN,NaN,NaN,NaN,359.379221,NaN,NaN,NaN,NaN,NaN,NaN
min,3.136000e+03,NaN,NaN,NaN,NaN,0.100000,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.216621e+06,NaN,NaN,NaN,NaN,3.600000,NaN,NaN,NaN,NaN,NaN,NaN
50%,2.126117e+06,NaN,NaN,NaN,NaN,5.700000,NaN,NaN,NaN,NaN,NaN,NaN
75%,4.172196e+06,NaN,NaN,NaN,NaN,10.600000,NaN,NaN,NaN,NaN,NaN,NaN


In [423]:
# df[df.CONCEPT_CD=='ICD9:628.9'].TVAL_CHAR.value_counts().to_frame()
# # PRIMARY 	1638
# # PROBLEM 	759
# # SECONDARY 	605
# # ADMITTING 	82

# df[df.CONCEPT_CD=='HDID:7177'].TVAL_CHAR.value_counts().to_frame()
# df[df.CONCEPT_CD=='SVC:BMC:OB/GYN-FAMILY PLANNING'].TVAL_CHAR.value_counts().to_frame()
# df[df.CONCEPT_CD=='ICD9:628.8'].TVAL_CHAR.value_counts().to_frame()
# # PROBLEM 	256
# # PRIMARY 	235
# # SECONDARY 	160
# # ADMITTING 	6

# df[df.CONCEPT_CD=='ICD9:628.0'].TVAL_CHAR.value_counts().to_frame()
# PRIMARY 	232
# SECONDARY 	125
# PROBLEM 	105
# ADMITTING 	1

# df[df.CONCEPT_CD=='HDID:3232'].TVAL_CHAR.value_counts().to_frame()

# df[df.CONCEPT_CD=='HDID:149'].NVAL_NUM.value_counts().to_frame()
# df[df.CONCEPT_CD=='HDID:301'].NVAL_NUM.value_counts().to_frame()
# df[df.CONCEPT_CD=='HDID:292'].NVAL_NUM.value_counts().to_frame()
# df[df.CONCEPT_CD=='HDID:3232'].NVAL_NUM.describe()#value_counts().to_frame()

df[df.PATIENT_NUM==71393]

,PATIENT_NUM,CONCEPT_CD,START_DATE,VALTYPE_CD,TVAL_CHAR,NVAL_NUM,CONCEPT_PATH,NAME_CHAR,SEX_CD,LANGUAGE_CD,RACE_CD,ZIP_CD
958,71393,HDID:292,2001-04-03 15:08:00,N,E,0.6,"\HDID2\Basic\Dx\Lab\Chemistry\progesterone, se...","progesterone, serum (HDID:292)",F,DEM|LANGUAGE:engl,DEM|RACE:black,2136
961,71393,HDID:7177,2001-12-15 09:48:22,T,neg,NaN,"\HDID2\Basic\Hx\PMH\infertility, hx of (HDID:7...","infertility, hx of (HDID:7177)",F,DEM|LANGUAGE:engl,DEM|RACE:black,2136
962,71393,HDID:2578,2006-06-16 14:42:39,T,neg,NaN,\HDID2\Basic\Dx\Lab\Chemistry\human chorionic ...,"human chorionic gonadotropin, urine, qualitati...",F,DEM|LANGUAGE:engl,DEM|RACE:black,2136


In [424]:
# df[df.CONCEPT_CD=='HDID:3232']#.TVAL_CHAR.value_counts().to_frame()

In [425]:
# #patient=df[df.PATIENT_NUM==1344]
# patient=df[df.PATIENT_NUM==217]
# patient.CONCEPT_CD.unique()

# sum(~df[df.PATIENT_NUM==1344].CONCEPT_CD.isin( ['HDID:2578','HDID:50588']))

# sum(~df[df.PATIENT_NUM==1104558].CONCEPT_CD.isin( ['HDID:2578','HDID:50588']))

#'FALSE' in patient.CONCEPT_CD.isin( ['HDID:2578','HDID:50588'])

# generate last preg to be predicted

In [426]:
keep_row=[]
for id in df.PATIENT_NUM.unique():#[0:1]:#[0:1]:
    patient=df[df.PATIENT_NUM==id].iloc[-1,:]
# #     print(patient)
#     k=len(patient)#RECORD_ID
    keep_row.append(df[df.PATIENT_NUM==id].index[-1])   
print(len(keep_row))

5712


In [427]:
df.loc[keep_row,:].TVAL_CHAR.value_counts().to_frame()#.describe()

,TVAL_CHAR
neg,4824
pos,888


In [428]:
# df.loc[keep_row,:]

In [429]:
df_preg=df.loc[keep_row,['PATIENT_NUM','START_DATE','TVAL_CHAR','LANGUAGE_CD','RACE_CD','ZIP_CD']]

df_preg.index=range(df_preg.shape[0])

# df_preg

# generate factors

In [430]:
x_df = df.index.isin(keep_row)
df_no_last=df[~x_df]

# df_no_last

df_no_last.CONCEPT_CD.value_counts().to_frame()

# df_no_last.CONCEPT_CD.unique()#value_counts().to_frame().index

,CONCEPT_CD
HDID:2578,3579
HDID:149,3571
HDID:301,2677
HDID:7177,1637
ICD9:628.9,1026
HDID:292,728
SVC:BMC:OB/GYN-FAMILY PLANNING,507
HDID:3232,346
ICD9:628.8,328
HDID:50588,255


In [431]:
# df_no_last_CD=df_no_last.loc[df_no_last.CONCEPT_CD==CD,:]
df_no_last['TVAL_NVAL']=''
df_no_last.loc[df_no_last.loc[:,'VALTYPE_CD']=='T','TVAL_NVAL']=df_no_last.loc[df_no_last.loc[:,'VALTYPE_CD']=='T','TVAL_CHAR']
df_no_last.loc[df_no_last.loc[:,'VALTYPE_CD']=='N','TVAL_NVAL']=df_no_last.loc[df_no_last.loc[:,'VALTYPE_CD']=='N','NVAL_NUM']


F:\Users\wudai.BMC\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [432]:
# df_no_last.loc[df_no_last.CONCEPT_CD==CD,'TVAL_NVAL'].values

# df_CD.loc[df_CD.PATIENT_NUM.isin(df_no_last.loc[df_no_last.CONCEPT_CD==CD,'PATIENT_NUM']),CD].unique()

df_CD=df_preg[['PATIENT_NUM']]

for CD in df_no_last.CONCEPT_CD.unique():#[0:1]
    print(CD)
    df_CD[CD]=''
    df_CD.loc[df_CD.PATIENT_NUM.isin(df_no_last.loc[df_no_last.CONCEPT_CD==CD,'PATIENT_NUM']),CD]=df_no_last.loc[df_no_last.CONCEPT_CD==CD,'TVAL_NVAL'].values
#     keep_row.append(patient[patient.CONCEPT_CD==CD].index[-1])

# df_CD

HDID:149
HDID:301


F:\Users\wudai.BMC\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


HDID:2578
HDID:292
HDID:7177
HDID:50588
HDID:3232
ICD9:628.9
SVC:BMC:OB/GYN-FAMILY PLANNING
ICD9:628.8
ICD9:628.0


In [433]:
df_all=pd.merge(df_preg,df_CD, on='PATIENT_NUM')

print(df_all.shape)
df_all.describe(include='all')

(5712, 17)


,PATIENT_NUM,START_DATE,TVAL_CHAR,LANGUAGE_CD,RACE_CD,ZIP_CD,HDID:149,HDID:301,HDID:2578,HDID:292,HDID:7177,HDID:50588,HDID:3232,ICD9:628.9,SVC:BMC:OB/GYN-FAMILY PLANNING,ICD9:628.8,ICD9:628.0
count,5.712000e+03,5712,5712,5712,5712,5711,5712,5712,5712,5712,5712,5712,5712,5712,5712,5712,5712
unique,NaN,5712,2,25,12,322,439,399,3,259,7,3,205,5,5,5,5
top,NaN,2013-02-01 14:40:07,neg,DEM|LANGUAGE:engl,DEM|RACE:black,2124,,,neg,,,,,,,,
freq,NaN,1,4824,4296,3398,528,2141,3035,3031,4984,4075,5457,5366,4686,5205,5384,5485
mean,2.902511e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.384046e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,3.136000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.213448e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2.103122e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,4.145596e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [435]:
df_all.to_csv('df_preg_hormone_last.csv', index=False)